In [1]:
import pandas as pd
import glob
import os
import numpy as np
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

# Reading data-set
# for Google CoLab
#from google.colab import drive
#drive.mount('/Dataset/')
#filename = r"/Dataset/MyDrive/Dataset/Uber_Dataset/prepared_data-set/month_prepared_dataset.csv"
filename = r".\Outputs\Prepared_data\month_prepared_dataset.csv"
################################################################################
# Reading data-set CSV file
data = pd.read_csv(filename, index_col=None, header=0)
del data["Unnamed: 0"]
data = data.reindex(columns=['lng_s', 'lat_s', 'lng_d', 'lat_d', 'month', 'distance', 'mean_travel_time', 'standard_deviation_travel_time'])
# Truncate Data
#data = data.truncate(before = 0, after = 499999)

# Labels are the values we want to predict
labels = data.filter(['mean_travel_time', 'standard_deviation_travel_time'], axis=1)
# Remove the labels from the features
features = data.filter(['lng_s', 'lat_s', 'lng_d', 'lat_d', 'month', 'distance'], axis=1)

# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.1, random_state = 0)

train_labels_margin = train_labels.filter(['standard_deviation_travel_time'], axis=1)
train_labels = train_labels.filter(['mean_travel_time'], axis=1)
test_labels_margin = test_labels.filter(['standard_deviation_travel_time'], axis=1)
test_labels = test_labels.filter(['mean_travel_time'], axis=1)

# Printing data-frame info
print("Data:")
print("Data shape is: ", data.shape)
print("Data columns are: ", data.columns)
#print("Data types are: ", data.dtypes)
print()

# Training labels matrix
print("Training labels Matrix:")
print("Training labels shape is: ", train_labels.shape)
print("Training labels columns are: ", train_labels.columns)
print()

# Training features matrix
print("Training features Matrix:")
print("Training features shape is: ", train_features.shape)
print("Training features columns are: ", train_features.columns)
print()

# Train labels margin matrix
print("Training labels margin Matrix:")
print("Training labels margin shape is: ", train_labels_margin.shape)
print("Training labels margin columns are: ", train_labels_margin.columns)
print()

# Testing labels matrix
print("Testing labels Matrix:")
print("Testing labels shape is: ", test_labels.shape)
print("Testing labels columns are: ", test_labels.columns)
print()

# Testing features matrix
print("Testing features Matrix:")
print("Testing features shape is: ", test_features.shape)
print("Testing features columns are: ", test_features.columns)

# Testing labels margin matrix
print("Testing labels margin Matrix:")
print("Testing labels margin shape is: ", test_labels_margin.shape)
print("Testing labels margin columns are: ", test_labels_margin.columns)

Data:
Data shape is:  (1966163, 8)
Data columns are:  Index(['lng_s', 'lat_s', 'lng_d', 'lat_d', 'month', 'distance',
       'mean_travel_time', 'standard_deviation_travel_time'],
      dtype='object')

Training labels Matrix:
Training labels shape is:  (1769546, 1)
Training labels columns are:  Index(['mean_travel_time'], dtype='object')

Training features Matrix:
Training features shape is:  (1769546, 6)
Training features columns are:  Index(['lng_s', 'lat_s', 'lng_d', 'lat_d', 'month', 'distance'], dtype='object')

Training labels margin Matrix:
Training labels margin shape is:  (1769546, 1)
Training labels margin columns are:  Index(['standard_deviation_travel_time'], dtype='object')

Testing labels Matrix:
Testing labels shape is:  (196617, 1)
Testing labels columns are:  Index(['mean_travel_time'], dtype='object')

Testing features Matrix:
Testing features shape is:  (196617, 6)
Testing features columns are:  Index(['lng_s', 'lat_s', 'lng_d', 'lat_d', 'month', 'distance'], dtype=

In [2]:
# random forest for making predictions for regression
from sklearn.ensemble import RandomForestRegressor
import time
T = time.time()
#model = RandomForestRegressor(n_estimators = 100, random_state=0, oob_score=True)
model = RandomForestRegressor(n_estimators = 100, random_state=0)
# fit the model on the whole dataset
model.fit(train_features, train_labels.values.ravel())
print("Training time is: " + str(round(time.time() - T, 2)) + " seconds.")

Training time is: 939.37 seconds.


In [3]:
from sklearn.metrics import r2_score
# Use the forest's predict method on the test data
predictions = model.predict(test_features)

# Same Error Function
#print("Accuracy: " + str(model.score(test_features, test_labels.values)*100) + "%")
print("Accuracy: " + str(r2_score(test_labels, predictions) * 100) + "%")

Accuracy: 98.63725268034858%


In [2]:
import joblib
joblib.dump(model, r".\Weights\Monthly\Monthly_Trained.joblib", compress=3)
#joblib.dump(model, "Monthly_Trained.joblib", compress=3)
#!cp Monthly_Trained.joblib "/Dataset/MyDrive/Dataset/Uber_Dataset/Trained_Model/Monthly_Trained.joblib"

NameError: name 'model' is not defined

In [3]:
# random forest for making predictions for regression
from sklearn.ensemble import RandomForestRegressor
import time
# Training Standard Deviation Time
T = time.time()
# define the model
#model = RandomForestRegressor(n_estimators = 100, random_state=0, oob_score=True)
model_SD = RandomForestRegressor(n_estimators = 100, random_state=0)
# fit the model on the whole dataset
model_SD.fit(train_features, train_labels_margin.values.ravel())
print("Training time for Standard Deviation is: " + str(round(time.time() - T, 2)) + " seconds.")

Training time for Standard Deviation is: 1305.95 seconds.


In [4]:
from sklearn.metrics import r2_score
# Use the forest's predict method on the test data
predictions_SD = model_SD.predict(test_features)

# Same Error Function
#print("Accuracy: " + str(model.score(test_features, test_labels.values)*100) + "%")
print("Accuracy: " + str(r2_score(test_labels_margin, predictions_SD) * 100) + "%")

Accuracy: 85.77579372512567%


In [5]:
import joblib
joblib.dump(model_SD, r".\Weights\Monthly\Monthly_SD_Trained.joblib", compress=3)
#joblib.dump(model, "Monthly_SD__Trained.joblib", compress=3)
#!cp Monthly_SD__Trained.joblib "/Dataset/MyDrive/Dataset/Uber_Dataset/Trained_Model/Monthly_SD_Trained.joblib"

['.\\Weights\\Monthly\\Monthly_SD_Trained.joblib']